## Planificación de la producción: Agencia Espacial Europea

V1.0, Julio 23

#### Problema CQM c/ reformulación Qubo

La Agencia Espacial Europea contará con un presupuesto de 2’4 millones de euros para
financiar misiones sobre Observación de la Tierra y programas de Transporte Espacial.

- Cada misión supone una inversión de 200.000 euros y cada programa 100.000 euros. 

- El número de Misiones debe de ser al menos 4, pero no más de la mitad del de Programas.

- Se estima que el retorno económico es del 60% del invertido en Misiones y del 40% en Programas


<b>¿cuántas misiones y programas se deben llevar a cabo para maximizar el retorno</b>


<b>Planteamiento algebraico:</b>

    - x, y ; variables  Misiones y  Programas

    - Función objetivo 1: minimizar{F(x,y)=-0'6*x - 0'4*y}
    

- Restricciones:

    - 2x1 + y <= 24
    - x >= 4
    - x - 0.5y <=0 





In [4]:
# Recursos
import dimod
from dimod import ConstrainedQuadraticModel, ExactCQMSolver, Integer
from dwave.system import DWaveSampler, EmbeddingComposite

# Funciones auxiliares

def sol_factibles(sampleset,n=None):
    # Resultados
    
    subset=sampleset.filter(lambda s: s.is_feasible).aggregate()
    
    if not n:
        print(f"\nLas {len(subset)} soluciones factibles al problema son:\n")
    else:
        print(f"\nImprimiendo las {n}/{len(subset)} primeras soluciones factibles:\n")
    
    print(subset.slice(n))
    
    
def sol_problema(result):
    # Agregación de los n reads
    samples = []
    ocurrencias = []

    for s in result.data():
        samples.append(invert(s.sample))
        ocurrencias.append(s.num_occurrences)

    sampleset = dimod.SampleSet.from_samples_cqm(samples,cqm,num_occurrences=ocurrencias)
    return(sampleset)

#### Caso simple

Obtener dimensionado surtidos atendiendo al criterio de maximizar el importe de venta 

In [6]:
## Definición del problema CQM

x = Integer('x',upper_bound=24)
y = Integer('y',upper_bound=24)

cqm = ConstrainedQuadraticModel()


# Función objetivo: Maximizar (ventas-costes)o minimizar (-ventas+costes)

cqm.set_objective(-0.6*x - 0.4*y)

# Restricciones

cqm.add_constraint(2*x + y <= 24, "Presupuesto: 2*x + y <= 24")
cqm.add_constraint(x >= 4,"Misiones mínimas: x>=4")
cqm.add_constraint(x - 2*y <= 0,"Misiones/Programas x -0.5y <=0")
print("Planteamiento problema CQM")
print("==========================")
print(cqm)

Planteamiento problema CQM
Constrained quadratic model: 2 variables, 3 constraints, 7 biases

Objective
  -0.6*Integer('x') - 0.4*Integer('y')

Constraints
  Presupuesto: 2*x + y <= 24: 2*Integer('x') + Integer('y') <= 24.0
  Misiones mínimas: x>=4: Integer('x') >= 4.0
  Misiones/Programas x -0.5y <=0: Integer('x') - 2*Integer('y') <= 0.0

Bounds
  0.0 <= Integer('x') <= 24.0
  0.0 <= Integer('y') <= 24.0



### Solución por fuerza bruta

In [7]:
result = ExactCQMSolver().sample_cqm(cqm)

In [8]:
solCPU=sol_factibles(result,10)


Imprimiendo las 10/51 primeras soluciones factibles:

  x  y energy num_oc. is_sat. is_fea.
0 4 16   -8.8       1 arra...    True
1 5 14   -8.6       1 arra...    True
2 6 12   -8.4       1 arra...    True
3 4 15   -8.4       1 arra...    True
4 5 13   -8.2       1 arra...    True
5 7 10   -8.2       1 arra...    True
6 8  8   -8.0       1 arra...    True
7 6 11   -8.0       1 arra...    True
8 4 14   -8.0       1 arra...    True
9 7  9   -7.8       1 arra...    True
['INTEGER', 10 rows, 10 samples, 2 variables]



<b>Resultados:</b>

- Misiones: 4

- Programas: 16

- Coste: 2.4M


### Solución con QPU

In [13]:
#reformulación qubo y sampling

fl=20 # factor penalizador de lagrange

qubo, invert = dimod.cqm_to_bqm(cqm, lagrange_multiplier = fl)

sampler = EmbeddingComposite(DWaveSampler())
result = sampler.sample(qubo, num_reads=2000)

In [14]:
sampleset=sol_problema(result)
solQPU=sol_factibles(sampleset,5)


Imprimiendo las 5/50 primeras soluciones factibles:

  x  y energy num_oc. is_sat. is_fea.
0 4 16   -8.8       1 arra...    True
1 5 14   -8.6       1 arra...    True
2 4 15   -8.4       3 arra...    True
3 6 12   -8.4       6 arra...    True
4 7 10   -8.2       5 arra...    True
['INTEGER', 5 rows, 16 samples, 2 variables]


### <b>Analizando los resultados del sampler QPU</b>

- Se ha formulado un problema Qubo desde un problema CQM

- El espacio de soluciones factibles es igual al de fuerza bruta

- Se han replicado los valores óptimos obtenidos por fuerza bruta

- num_reads=2000 solo produce una ocurrencia del valor óptimo, subir a 3000.

